<a href="https://colab.research.google.com/github/Talamanj08/Project-4/blob/main/Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression

In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [ ]:
github_csv_url = "https://github.com/Talamanj08/Project-4/raw/main/imdb_movies_clean.csv"
df = pd.read_csv(github_csv_url)
df.head()


,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,John Wick: Chapter 4,3/23/2023,80,"Action,Â Thriller,Â Crime","With the price on his head ever increasing, Jo...","Keanu Reeves, John Wick, Donnie Yen, Caine, Bi...",John Wick: Chapter 4,Released,English,100000000,351349364.0,AU
1,John Wick: Chapter 2,5/18/2017,73,"Action,Â Thriller,Â Crime",John Wick is forced out of retirement by a for...,"Keanu Reeves, John Wick, Common, Cassian, Ian ...",John Wick: Chapter 2,Released,English,40000000,171539887.0,AU
2,John Wick: Chapter 3 - Parabellum,5/16/2019,74,"Action,Â Thriller,Â Crime",Super-assassin John Wick returns with a $14 mi...,"Keanu Reeves, John Wick, Halle Berry, Sofia, I...",John Wick: Chapter 3 - Parabellum,Released,English,55000000,326709727.0,AU
3,Scream VI,3/9/2023,73,"Horror,Â Mystery,Â Thriller","Following the latest Ghostface killings, the f...","Melissa Barrera, Sam Carpenter, Jenna Ortega, ...",Scream VI,Released,English,35000000,167273149.0,AU
4,John Wick,11/26/2014,74,"Action,Â Thriller",Ex-hitman John Wick comes out of retirement to...,"Keanu Reeves, John Wick, Michael Nyqvist, Vigg...",John Wick,Released,English,30000000,75958800.0,AU


In [ ]:
df["country"].unique()

array(['AU', 'US', 'GB', 'JP', 'CA', 'PT'], dtype=object)

In [ ]:
clean_df = df.drop(columns=["overview","crew","orig_title","status","orig_lang"])
clean_df.head()

,names,date_x,score,genre,budget_x,revenue,country
0,John Wick: Chapter 4,3/23/2023,80,"Action,Â Thriller,Â Crime",100000000,351349364.0,AU
1,John Wick: Chapter 2,5/18/2017,73,"Action,Â Thriller,Â Crime",40000000,171539887.0,AU
2,John Wick: Chapter 3 - Parabellum,5/16/2019,74,"Action,Â Thriller,Â Crime",55000000,326709727.0,AU
3,Scream VI,3/9/2023,73,"Horror,Â Mystery,Â Thriller",35000000,167273149.0,AU
4,John Wick,11/26/2014,74,"Action,Â Thriller",30000000,75958800.0,AU


In [ ]:
clean_df["Profit"] = ((clean_df["revenue"] - clean_df["budget_x"]) / clean_df["revenue"]) *100
clean_df.head(20)

,names,date_x,score,genre,budget_x,revenue,country,Profit
0,John Wick: Chapter 4,3/23/2023,80,"Action,Â Thriller,Â Crime",100000000,3.513494e+08,AU,71.538301
1,John Wick: Chapter 2,5/18/2017,73,"Action,Â Thriller,Â Crime",40000000,1.715399e+08,AU,76.681808
2,John Wick: Chapter 3 - Parabellum,5/16/2019,74,"Action,Â Thriller,Â Crime",55000000,3.267097e+08,AU,83.165484
3,Scream VI,3/9/2023,73,"Horror,Â Mystery,Â Thriller",35000000,1.672731e+08,AU,79.076140
4,John Wick,11/26/2014,74,"Action,Â Thriller",30000000,7.595880e+07,AU,60.504905
5,Scream,1/13/2022,67,"Horror,Â Mystery,Â Thriller",24000000,1.388748e+08,AU,82.718246
6,Scream,1/13/2022,67,"Horror,Â Mystery,Â Thriller",15000000,1.730466e+08,AU,91.331817
7,Avengers: Infinity War,4/25/2018,83,"Adventure,Â Action,Â Science Fiction",300000000,2.048360e+09,AU,85.354135
8,Jeepers Creepers: Reborn,9/22/2022,57,"Horror,Â Thriller,Â Mystery",5000000,5.954938e+06,AU,16.036070
9,Shrek,6/21/2001,77,"Animation,Â Comedy,Â Fantasy,Â Adventure,Â Family",50000000,4.918128e+08,AU,89.833530


In [ ]:
avg_profit = clean_df["Profit"].median()
print(avg_profit)

80.64875712578879


In [ ]:
success_list = []

for index, row in clean_df.iterrows():
    if row["Profit"] > 80.64875712578879:
        success_list.append("Yes")
    else:
        success_list.append("No")

clean_df["Success"] = success_list

In [ ]:
clean_df.head(20)

,names,date_x,score,genre,budget_x,revenue,country,Profit,Success
0,John Wick: Chapter 4,3/23/2023,80,"Action,Â Thriller,Â Crime",100000000,3.513494e+08,AU,71.538301,No
1,John Wick: Chapter 2,5/18/2017,73,"Action,Â Thriller,Â Crime",40000000,1.715399e+08,AU,76.681808,No
2,John Wick: Chapter 3 - Parabellum,5/16/2019,74,"Action,Â Thriller,Â Crime",55000000,3.267097e+08,AU,83.165484,Yes
3,Scream VI,3/9/2023,73,"Horror,Â Mystery,Â Thriller",35000000,1.672731e+08,AU,79.076140,No
4,John Wick,11/26/2014,74,"Action,Â Thriller",30000000,7.595880e+07,AU,60.504905,No
5,Scream,1/13/2022,67,"Horror,Â Mystery,Â Thriller",24000000,1.388748e+08,AU,82.718246,Yes
6,Scream,1/13/2022,67,"Horror,Â Mystery,Â Thriller",15000000,1.730466e+08,AU,91.331817,Yes
7,Avengers: Infinity War,4/25/2018,83,"Adventure,Â Action,Â Science Fiction",300000000,2.048360e+09,AU,85.354135,Yes
8,Jeepers Creepers: Reborn,9/22/2022,57,"Horror,Â Thriller,Â Mystery",5000000,5.954938e+06,AU,16.036070,No
9,Shrek,6/21/2001,77,"Animation,Â Comedy,Â Fantasy,Â Adventure,Â Family",50000000,4.918128e+08,AU,89.833530,Yes


In [ ]:
clean_df["Success"].value_counts()

No     134
Yes    133
Name: Success, dtype: int64

In [ ]:
def format_scientific_to_decimal(num):
    return f'{num:.2f}'
clean_df["revenue"] = clean_df["revenue"].apply(format_scientific_to_decimal)
clean_df.head(20)

,names,date_x,score,genre,budget_x,revenue,country,Profit,Success
0,John Wick: Chapter 4,3/23/2023,80,"Action,Â Thriller,Â Crime",100000000,351349364.00,AU,71.538301,No
1,John Wick: Chapter 2,5/18/2017,73,"Action,Â Thriller,Â Crime",40000000,171539887.00,AU,76.681808,No
2,John Wick: Chapter 3 - Parabellum,5/16/2019,74,"Action,Â Thriller,Â Crime",55000000,326709727.00,AU,83.165484,Yes
3,Scream VI,3/9/2023,73,"Horror,Â Mystery,Â Thriller",35000000,167273149.00,AU,79.076140,No
4,John Wick,11/26/2014,74,"Action,Â Thriller",30000000,75958800.00,AU,60.504905,No
5,Scream,1/13/2022,67,"Horror,Â Mystery,Â Thriller",24000000,138874789.00,AU,82.718246,Yes
6,Scream,1/13/2022,67,"Horror,Â Mystery,Â Thriller",15000000,173046640.00,AU,91.331817,Yes
7,Avengers: Infinity War,4/25/2018,83,"Adventure,Â Action,Â Science Fiction",300000000,2048359754.00,AU,85.354135,Yes
8,Jeepers Creepers: Reborn,9/22/2022,57,"Horror,Â Thriller,Â Mystery",5000000,5954938.00,AU,16.036070,No
9,Shrek,6/21/2001,77,"Animation,Â Comedy,Â Fantasy,Â Adventure,Â Family",50000000,491812794.00,AU,89.833530,Yes
